In [6]:
from mur_wind_prev import  get_wind_prev_pivot

In [2]:
ZONES = ("22Bolo", "24Ama1", "26Fanh", "27Arr1", "29Sob2", "44Milagre", "48Vale_de", "60Maravil", "61MaraviII", "68Jogui")
HIST_NUM_YEARS = 4

In [4]:
df = get_wind_prev_pivot(ZONES, HIST_NUM_YEARS)

In [152]:
import tensorflow as tf
import tensorflow_probability as tfp

tfd = tfp.distributions

In [148]:
num_timesteps, num_series = df.shape
num_predictors = 5 
# def build_mbsts_model(num_timesteps, num_series, num_predictors):

In [149]:
# Example predictors
predictors = tf.Variable(tf.random.normal([num_timesteps, num_predictors]), name='predictors')

In [150]:
# Define priors for the model parameters
mu = tf.Variable(tf.zeros([num_series]), name='mu')
sigma = tfp.util.TransformedVariable(tf.eye(num_series), tfp.bijectors.FillScaleTriL(), name='sigma')

In [153]:
# Trend component
# trend = tfd.Normal(loc=tf.zeros([num_timesteps, num_series]), scale=1.0, name='trend')

 # Sample from the distributions for each component
trend_dist = tfd.Normal(loc=tf.zeros([num_timesteps, num_series]), scale=1.0)
trend = trend_dist.sample()

In [124]:
# Seasonal component
# seasonality = tfd.Normal(loc=tf.zeros([num_timesteps, num_series]), scale=1.0, name='seasonality')

seasonality_dist = tfd.Normal(loc=tf.zeros([num_timesteps, num_series]), scale=1.0)
seasonality = seasonality_dist.sample()

In [125]:
# Cyclical component
# cycle = tfd.Normal(loc=tf.zeros([num_timesteps, num_series]), scale=1.0, name='cycle')
cycle_dist = tfd.Normal(loc=tf.zeros([num_timesteps, num_series]), scale=1.0)
cycle = cycle_dist.sample()

In [126]:
# Regression component
# regression_coefficients = tfd.Normal(loc=tf.zeros([num_series, num_predictors]), scale=1.0,name='regression_coefficients')
# predictors = tf.Variable(tf.zeros([num_timesteps, num_predictors]), name='predictors')

regression_coefficients_dist = tfd.Normal(loc=tf.zeros([num_series, num_predictors]), scale=1.0)
regression_coefficients = regression_coefficients_dist.sample()


In [158]:
# Observation model
observation_model = (
        mu 
        + trend 
        + seasonality 
        + cycle 
        + tf.linalg.matmul(predictors, regression_coefficients, transpose_b=True)
)

In [159]:
# Add noise
model = tfd.MultivariateNormalTriL(loc=observation_model, scale_tril=sigma)

    # return y

In [160]:
# Define the log probability of the model
log_model_prob = model.log_prob(df.values)

### Fit the model using MCMC

In [161]:
# Prepare data for the model
# observations = df.values

In [162]:
def target_log_prob_fn(log_model=log_model_prob):
    # return tf.reduce_sum(model.log_prob(observations))
    return log_model

In [163]:
# Define the HMC transition kernel
nuts_kernel = tfp.mcmc.HamiltonianMonteCarlo(
    target_log_prob_fn=target_log_prob_fn,
    step_size=0.1,
    num_leapfrog_steps=3
)

In [164]:
# Run the MCMC chain
num_results = 500
num_burnin_steps = 300


In [165]:
@tf.function
def run_chain(model, num_results, num_burnin_steps, nuts_kernel):
    # Initial state for MCMC
    initial_state = model.trainable_variables

    return tfp.mcmc.sample_chain(
        num_results=num_results,
        num_burnin_steps=num_burnin_steps,
        current_state=initial_state,
        kernel=nuts_kernel,
        trace_fn=lambda current_state, kernel_results: kernel_results.is_accepted
    )

In [166]:
# Execute the MCMC chain
samples, is_accepted = run_chain(model, num_results, num_burnin_steps, nuts_kernel)

In [ ]:
import matplotlib.pyplot as plt

# Extract posterior means
posterior_mu = samples[0].numpy()

plt.figure(figsize=(10, 6))
for i in range(num_series):
    plt.plot(posterior_mu[:, i], label=f'Time Series {i+1}')
plt.legend()
plt.title('Posterior Means of Time Series')
plt.show()
